<a href="https://colab.research.google.com/github/arjunramesh99/EE371Q-Project/blob/main/Face_Emoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
from google.colab import files
files.upload()

!tar -xzvf RAF_Sorted.tar.gz
%cd /content/RAF_Sorted

In [ ]:
%cd /content/drive/ML/RAF_Sorted

In [ ]:
!ls test/0 -1 | wc -l
!ls test/1 -1 | wc -l
!ls test/2 -1 | wc -l
!ls test/3 -1 | wc -l
!ls test/4 -1 | wc -l
!ls test/5 -1 | wc -l
!ls test/6 -1 | wc -l

!ls train/0 -1 | wc -l
!ls train/1 -1 | wc -l
!ls train/2 -1 | wc -l
!ls train/3 -1 | wc -l
!ls train/4 -1 | wc -l
!ls train/5 -1 | wc -l
!ls train/6 -1 | wc -l

329
74
160
1185
478
162
680
1290
281
717
4772
1982
705
2524


In [ ]:
!ls

In [ ]:
from keras.models import Model, load_model, Sequential
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.vgg16 import VGG16
from matplotlib import pyplot as plt
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.models import Model, load_model, Sequential
from matplotlib import pyplot as plt
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from pathlib import Path

'''
    Model Parameters
        Number of Images (before augmentation):
          Train : 12271
          Test  : 3068
'''
# ----------------------------------- #
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 8
MAX_TRAIN_IMAGE_COUNT = 12271
MAX_TEST_IMAGE_COUNT = 3068
EPOCHS = 200
IMG_HEIGHT = 100
IMG_WIDTH = 100
# ----------------------------------- #


class Resnet50_Trainer:
  
  def __init__(self, train_dir, test_dir):
    '''
      Initializes train and test directories for the trainer
    '''
    self._train_dir = train_dir
    self._test_dir = test_dir
      
  
  def __init_data_generators(self):
    '''
      Initializes image data generators for yielding real-time augmented data
    '''
    train_datagen = image.ImageDataGenerator(rescale = 1./255, zoom_range = 0.1,
                                              brightness_range = [0.6, 1.4], horizontal_flip = True, validation_split=0.2)
    test_datagen = image.ImageDataGenerator()#rescale = 1./255, shear_range = 0.2, zoom_range = 0.3,
                                            #  horizontal_flip = True, vertical_flip = True, rotation_range = 90)
       
    ''' Using .flow_from_directory '''
    self.train_generator = train_datagen.flow_from_directory(self._train_dir, target_size = (IMG_HEIGHT, IMG_WIDTH), subset="training",
                                                               batch_size = TRAIN_BATCH_SIZE, class_mode = 'categorical', shuffle = True)
                                                                   
    self.validation_generator = train_datagen.flow_from_directory(self._train_dir, target_size = (IMG_HEIGHT, IMG_WIDTH), subset="validation",
                                                               batch_size = TRAIN_BATCH_SIZE, class_mode = 'categorical', shuffle = True)
    
    self.test_generator = test_datagen.flow_from_directory(self._test_dir, target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                               batch_size = TEST_BATCH_SIZE, class_mode = 'categorical', shuffle = True)
                                                               
                                                               
        
    print("Image Data Generators have been instantiated!\n")
    print(self.train_generator.class_indices, self.validation_generator.class_indices, self.test_generator.class_indices)
          
          
  def __init_model_architecture(self):
    ''' 
        Define ResNet50 model architecture, trained with "freeze_count" layers frozen
    '''
    frz_ct = 10

    model = VGG16(include_top = False, weights = 'imagenet', input_shape = (IMG_HEIGHT, IMG_WIDTH, 3), pooling = 'max')
    for layer in model.layers[:frz_ct]:
      layer.trainable = False
    #model.trainable = False
    '''
    # Model Architecture
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(576, 432, 3), data_format = "channels_last"))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size = (3, 3)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (5, 5), activation='relu', data_format = "channels_last"))
    model.add(Conv2D(64, (7, 7), activation='relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(64, (10, 10), activation='relu', data_format = "channels_last"))
    model.add(Conv2D(64, (13, 13), activation='relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))    
    '''
    # Adding final layers
    x = model.output
    x = Dense(4096, activation = 'relu')(x) # 1024
    x = Dense(2048, activation = 'relu')(x)
    predictions = Dense(7, activation = 'softmax')(x)

    self.model_final = Model(inputs = model.input, outputs = predictions)
    #opt = optimizers.SGD(lr = 0.0001)  # lr = 0.0001
    self.model_final.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    print("Resnet50 Initialization Successful!")
    
    
  def __model_train(self):
    '''
         Fit the model to training data with certain loss and saves best model using callbacks
    '''
    #self.model_final = load_model('drive/best_model.h5')
    class_weight = {0: 3,
                    1: 16,
                    2: 6,
                    3: 1,
                    4: 2.4,
                    5: 6,
                    6: 1.7}
    filepath = 'best_model_2048_frz_full.h5'
    checkpoint = ModelCheckpoint(filepath, monitor = 'val_accuracy', verbose = 1, save_best_only = True)
    print("Prepare for Training...")
    self.hist = self.model_final.fit(self.train_generator,
                                     steps_per_epoch = self.train_generator.samples // TRAIN_BATCH_SIZE,
                                     epochs = EPOCHS,
                                     class_weight = class_weight,
                                     validation_data = self.validation_generator,
                                     validation_steps = self.validation_generator.samples // TRAIN_BATCH_SIZE,
                                     callbacks = [checkpoint])

  def __model_eval(self):
      # Evaluate model on test data
    score = self.model_final.evaluate(self.test_generator, steps = MAX_TEST_IMAGE_COUNT // TEST_BATCH_SIZE, verbose = 1)
    print(score)
    '''
      # summarize history for accuracy
      plt.plot(self.hist.history['acc'])
      plt.plot(self.hist.history['val_acc'])
      plt.title('model accuracy')
      plt.ylabel('accuracy')
      plt.xlabel('epoch')
      plt.legend(['train', 'test'], loc='upper left')
      plt.show()
    '''
        
  def run_trainer(self):
    self.__init_data_generators()
    self.__init_model_architecture()
    self.__model_train()
    self.__model_eval()
        
        
if __name__ == '__main__':
  #ResnetTrainer.model_final.save('best_model_low_lr.h5')
  base_path = Path("/content/RAF_Sorted")
  train_path = base_path / 'train'
  test_path = base_path / 'test'
  ResnetTrainer = Resnet50_Trainer(train_path, test_path)
  ResnetTrainer.run_trainer()

In [ ]:
qq

In [ ]:
ResnetTrainer.model_final.summary()

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 106, 106, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 50, 50, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 50, 50, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________

In [ ]:
from google.colab import files
files.download('best_model_2048.h5')

# New Section